In [14]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import geopandas as gpd
import json
from shapely.geometry import shape, GeometryCollection
from rasterio.plot import show
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from utils import FeatureExtraction, feature_matching, decimal_coords
from exif import Image as exifimg
from PIL import Image

import shutil
import rasterio
import cv2
%matplotlib inline

In [15]:
# 가능한 font list 확인
import matplotlib.font_manager as fm
f = [f.name for f in fm.fontManager.ttflist]
# print(f)

plt.rc('font', family='Malgun Gothic')

In [27]:
img_folder = r'D:\original'
out_folder = r'E:\ToAWS2'

course_list = ['청도','포항','중문','시흥','거창','속리산','춘천']

course_ids = {
  '포항':'MGC001',
  '청도':'MGC002',
  '중문':'MGC003',
  '시흥':'MGC004',
  '거창':'MGC005',
  '속리산':'MGC006',
  '춘천':'MGC007'
}

folder_type = ['rgb','ndvi','thumb']

In [28]:
course_id = course_ids.get('중문')
course_id

'MGC003'

In [29]:
target_folders = glob.glob(os.path.join(out_folder,'**/*'))
target_folders = [x for x in target_folders if course_id in x]
target_folders

['E:\\ToAWS2\\MGC003\\20230910']

In [30]:
target_folder = target_folders[0]

In [31]:
target_date = os.path.split(target_folder)[-1]
course_id, target_date

('MGC003', '20230910')

In [32]:
dataJson_ = glob.glob(os.path.join(out_folder,course_id,target_date,'**/photo*.json'), recursive=True)[0]
dataJson_ 

'E:\\ToAWS2\\MGC003\\20230910\\photo.json'

In [33]:
with open(dataJson_, "r") as _json:
  dataJson= json.load(_json)

In [34]:
len(dataJson)

16961

In [35]:
dataJson[0]

{'id': '20230910131355_1264064138889_332512323889_92468000',
 'originalFileJPG': 'ta/nas/10_원시데이터(3사_분류대상)/2023_0910_중문골프클럽_데이터 1차 분류_완/중문골프클럽 잔디보식지역/2023_0910_중문골프클럽_잔디보식지역_0.5m/20230910_1313-1345_잔디보식지역/DJI_20230910131355_0001_D.JPG',
 'originalFileGrp': ['ta/nas/10_원시데이터(3사_분류대상)/2023_0910_중문골프클럽_데이터 1차 분류_완/중문골프클럽 잔디보식지역/2023_0910_중문골프클럽_잔디보식지역_0.5m/20230910_1313-1345_잔디보식지역/DJI_20230910131355_0001_MS_G.TIF',
  'ta/nas/10_원시데이터(3사_분류대상)/2023_0910_중문골프클럽_데이터 1차 분류_완/중문골프클럽 잔디보식지역/2023_0910_중문골프클럽_잔디보식지역_0.5m/20230910_1313-1345_잔디보식지역/DJI_20230910131355_0001_MS_NIR.TIF',
  'ta/nas/10_원시데이터(3사_분류대상)/2023_0910_중문골프클럽_데이터 1차 분류_완/중문골프클럽 잔디보식지역/2023_0910_중문골프클럽_잔디보식지역_0.5m/20230910_1313-1345_잔디보식지역/DJI_20230910131355_0001_MS_R.TIF',
  'ta/nas/10_원시데이터(3사_분류대상)/2023_0910_중문골프클럽_데이터 1차 분류_완/중문골프클럽 잔디보식지역/2023_0910_중문골프클럽_잔디보식지역_0.5m/20230910_1313-1345_잔디보식지역/DJI_20230910131355_0001_MS_RE.TIF'],
 'destFolder': '20230910/중문골프클럽 잔디보식지역/잔디보식지역_0',
 'GeoTagInfo': {'coords': [126.40641388888889,

In [36]:
import boto3
from decimal import Decimal
# Get the service resource.
dynamodb = boto3.resource('dynamodb', region_name='ap-northeast-2')
table = dynamodb.Table('dsnia_imagelabelsV2')
# table = dynamodb.Table('dsnia_imagelabels')

In [37]:
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError

with table.batch_writer(overwrite_by_pkeys=['metadata', 'id']) as batch:
  for index_, row_ in enumerate(dataJson):
    newrow = {}
    newrow ['id'] = row_['id']
    newrow ['metadata'] = course_id+target_date+row_['info']['area']+row_['info']['desc']
    newrow ['courseid'] = course_id
    newrow ['date'] = target_date
    newrow ['area'] = row_['info']['area']
    newrow ['desc'] = row_['info']['desc']
    newrow ['json'] = json.loads(json.dumps(row_), parse_float=Decimal)
    print(index_, end='\r')
    batch.put_item(
      Item= newrow
    )

In [49]:
courseid = course_id
date = target_date

response = table.query(IndexName='courseid-id-index', KeyConditionExpression=Key('courseid').eq(courseid),  FilterExpression=Attr('date').eq(date))
data =  response['Items']

while 'LastEvaluatedKey' in response:
    response = table.query(IndexName='courseid-id-index', KeyConditionExpression=Key('courseid').eq(courseid), 
     ExclusiveStartKey=response['LastEvaluatedKey'] ,FilterExpression=Attr('date').eq(date) )    
    # response_ = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    data.extend(response['Items'])

In [50]:
len(data)

7436

In [51]:
courseid, date

('MGC004', '20230815')

In [57]:
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError

courseid = course_id
date = target_date

area = "아세코벨리 1H"

desc = "광엽,화본과,민들래,바랭이,애기땅빈대,왕바랭이,이종잔디,조이시아그래스,사초과,방동사니_0"

response = table.query(KeyConditionExpression=Key('metadata').eq(courseid+date+area+desc))
data =  response['Items']

while 'LastEvaluatedKey' in response:
    response = table.query(KeyConditionExpression=Key('metadata').eq(courseid+date+area+desc),  ExclusiveStartKey=response['LastEvaluatedKey']  )    
    # response_ = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    data.extend(response['Items'])

In [58]:
len(data)

290